In [2]:
def ascii_encode(msg):

    lst = []
    for i in msg:
        lst.append(ord(i))

    return lst

#print(ascii_encode('hello world'))

In [3]:
def shift_ascii_numbers(numb_list, shift_val):
    '''
    we will restrict ourselves to the set of single printable characters from the ascii table. These range from 32 to 126 in the ascii table - a total of 95 characters. 
    
    To ensure no overflow after shifting, we first subtract 32 to take us to the range 0 to 94, make the shift modulo 95, then add 32 again to take us back to the suitable range. 
    '''

    shifted_numb_list =  []
    for i in numb_list:
        temp_val = ( (i -32 + shift_val) ) % 95 + 32
        shifted_numb_list.append(temp_val)
    return shifted_numb_list

#lst = shift_ascii_numbers([104, 101, 108, 108, 111, 32, 119, 111, 114, 108, 100], 77 )
#lst


In [4]:
def ascii_decode(msg):
    
    lst = []
    for i in msg:
        lst.append( chr(i))
        
    return lst

#output = ascii_decode([86, 83, 90, 90, 93, 109, 101, 93, 96, 90, 82])
#print(output)
#print(''.join(output))

In [5]:
def ascii_caesar_encrypt(message, key):
    
    #step 1: ascii encode
    ascii_message = ascii_encode(message)
    
    #step 2: step 2: shift ascii numbers
    shifted_ascii_message = shift_ascii_numbers(ascii_message, key)
    
    #step 3: ascii decode 
    encrypted_message = ascii_decode(shifted_ascii_message)
    
    #step 4: convert to string and return 
    return ''.join(encrypted_message)

#print(ascii_caesar_encrypt("hello world again", 23))

In [6]:
def ascii_caesar_decrypt(encrypted_message, key):
    
    #step 1: ascii encode
    ascii_message = ascii_encode(encrypted_message)
    
    #step 2: step 2: shift ascii numbers
    shifted_ascii_message = shift_ascii_numbers(ascii_message, -1*key)
    
    #step 3: ascii decode 
    decrypted_message = ascii_decode(shifted_ascii_message)
    
    #step 4: convert to string and return 
    return ''.join(decrypted_message)

In [7]:
def inputMessage(message_limit=50):

    message = input('Enter your message: ')
    while len(message) > message_limit:
        print('Your message is too long!')
        message = input('Enter your message: ')

    encryption_type = 0
    while (encryption_type not in ['1', '2']):
        encryption_type = input('Which encryption method would you like to use?\n (1. Ceasar cipher 2. Other): ')
    encryption_type = int(encryption_type)

    encryption_key = 0
    while (encryption_key not in [str(i) for i in range(1, 95)]):
        encryption_key = input('Enter an encryption key between 1 - 95: ')
    encryption_key = int(encryption_key)
    
    encryption_dir = ''
    while encryption_dir not in ['e','d']: 
        encryption_dir = input("Would you like to encrypt (e) or decrypt (d)?")
        
    return message, encryption_type, encryption_key, encryption_dir
    
#inputMessage()    


In [8]:
def get_user_encryption_type():

    encryption_type = 0
    while (encryption_type not in ['1', '2']):
        encryption_type = input('Which encryption method would you like to use?\n (1. Ceasar cipher 2. RSA): ')
    encryption_type = int(encryption_type)

    return encryption_type

def encryption_keys_needed():

    keys_needed = input("Do you need to generate public and private keys? Y/N")

    return keys_needed

def get_user_message(message_limit = 50):
    message = input('Enter your message: ')
    while len(message) > message_limit:
        print('Your message is too long!')
        message = input('Enter your message: ')

    return message

def get_encryption_key():
    encryption_key = 0
    while (encryption_key not in [str(i) for i in range(1, 95)]):
        encryption_key = input('Enter an encryption key between 1 - 95: ')
    encryption_key = int(encryption_key)
    return encryption_key

def get_encryption_dir():
    encryption_dir = ''
    while encryption_dir not in ['e','d']: 
        encryption_dir = input("Would you like to encrypt (e) or decrypt (d)?")
    return encryption_dir



In [ ]:
# main logic 

while True:
    encryption_type = get_user_encryption_type()
    message =  get_user_message()
    encryption_key = get_encryption_key()
    encryption_dir = get_encryption_dir()
    
    if encryption_type == 1:
        #the user wants to use the caeser cipher
        if encryption_dir == 'e':
            #the use wants to encrypt
            print(f"Your encrypted message using the caeser cipher and key {encryption_key} is:")
            print('\t', ascii_caesar_encrypt(message, encryption_key))
        else:
            #the user wants to decrypt
            print(f"Your decrypted message using the caeser cipher and key {encryption_key} is:")
            print(ascii_caesar_decrypt(message, encryption_key))
        
            
    
    elif encryption_type == 2:
        #the user want to use another encryption type
        pass
    
    
    another_message = input("Would you like to encrypt/decrypt another message? Y/N")
    if another_message == "N":
        break
    

KeyboardInterrupt: Interrupted by user